In [10]:
import numpy as np
import datetime
import os
import sklearn.metrics
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from scipy.stats import chi
from scipy import stats
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model, Sequential

In [11]:
############
# Functions
############

# Our 3d parameter space is the surface of a sphere centered at (1, 1, 1), with radius 1/sqrt(2)
# x, y coordinates are the x and y variances of our 2d Gaussian
# z coordinate determines the x-coordinate of the mean, mu = (z, 0)

def mean_gen(theta, phi):
    mu = 0.5 - (np.sqrt(2)/2) * np.cos(phi + np.pi / 4)
    return mu


def varx_gen(theta, phi):
    vx = 1.5 - (np.sqrt(2)/2) * np.cos(theta + np.pi / 4) * np.sin(phi + np.pi / 2)
    return vx


def vary_gen(theta, phi):
    vy = 1.5 - (np.sqrt(2)/2) * np.sin(theta + np.pi / 4) * np.sin(phi + np.pi / 2)
    return vy


# Generate n data
def spherical_data(n, thetas, phis, rand=1234):
    
    mx1, my1 = np.zeros(n), np.zeros(n)
    vx1, vy1 = np.ones(n), np.ones(n)
    
    mx2, my2 = mean_gen(thetas, phis), np.zeros(n)
    vx2, vy2 = varx_gen(thetas, phis), vary_gen(thetas, phis)
    
    x1, y1 = np.transpose(np.array([np.random.normal(mx1, vx1, size=n), 
                                    np.random.normal(my1, vy1, size=n), thetas, phis])), np.zeros(n)
    x2, y2 = np.transpose(np.array([np.random.normal(mx2, vx2, size=n), 
                                    np.random.normal(my2, vy2, size=n), thetas, phis])), np.ones(n)
    
    x, y = np.append(x1, x2, axis=0), np.append(y1, y2, axis=0)

    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = rand)
    
    return x_train, x_test, y_train, y_test

In [12]:
# Single Point on Sphere

n = 40000
rand_n = 12345
theta, phi = 3, 3
thetas, phis = theta * np.ones(n), phi * np.ones(n)
x_train, x_test, y_train, y_test = spherical_data(n, thetas, phis, rand_n)

# model = tf.keras.models.load_model('3dmodels/discrete_model_mth15_mph15')
model = tf.keras.models.load_model('3dmodels/discrete_model_th3_ph3')

In [13]:
for l in model.layers:
    l.trainable=False

inputs_hold = tf.keras.Input(shape=(1,))
simple_model = Dense(2, use_bias = False)(inputs_hold)
model3 = Model(inputs = inputs_hold, outputs = simple_model)

inputs = tf.keras.Input(shape=(2,))
inputs2 = tf.keras.layers.concatenate([inputs, model3(tf.ones_like(inputs)[:,0:1])])
hidden_layer_1 = model(inputs2)

model2 = Model(inputs = inputs, outputs = hidden_layer_1)
loss_fn = tf.keras.losses.BinaryCrossentropy()
model2.compile(loss=loss_fn, optimizer='Adam', )

model2.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 2)]          0                                            
__________________________________________________________________________________________________
tf.ones_like_1 (TFOpLambda)     (None, 2)            0           input_4[0][0]                    
__________________________________________________________________________________________________
tf.__operators__.getitem_1 (Sli (None, 1)            0           tf.ones_like_1[0][0]             
__________________________________________________________________________________________________
model_2 (Functional)            (None, 2)            2           tf.__operators__.getitem_1[0][0] 
____________________________________________________________________________________________

In [18]:
names = []
names.append('discrete_model_th5_ph5')
names.append('discrete_model_th5_ph3')
names.append('discrete_model_th4_ph3')
names.append('discrete_model_th3_ph3')
names.append('discrete_model_th2_ph3')
names.append('discrete_model_th0_ph0')
names.append('discrete_model_mth15_mph15')

l = len(names)

models = []
predictions = []
for i in range(l):
    models.append(tf.keras.models.load_model('3dmodels/' + names[i]))
    xr = models[i](x_train).numpy().transpose()[0]
    loss = np.sum(-y_train * np.log(xr) - (1-y_train) * np.log(1-xr)) / xr.shape[0]
    print(names[i], 'has loss', loss)

discrete_model_th5_ph5 has loss 0.7176607511820549
discrete_model_th5_ph3 has loss 0.6682014072554356
discrete_model_th4_ph3 has loss 0.6352690782713435
discrete_model_th3_ph3 has loss 0.5615279547481478
discrete_model_th2_ph3 has loss 0.5785765893505459
discrete_model_th0_ph0 has loss 0.6880507253501564
discrete_model_mth15_mph15 has loss 0.5869762585826247


In [16]:
coordinates = x_train[:, 0:2]
real_model = tf.keras.models.load_model('3dmodels/discrete_model_th3_ph3')
x_r = real_model(x_train).numpy().transpose()[0]

for i in range(10):
    model2.fit(coordinates, y_train, epochs=1,batch_size=100)
    print(i,"Fitted result: ", model2.trainable_weights[:][0][0])
    x = model2(coordinates).numpy().transpose()[0]
    print(np.sum(-y_train * np.log(x) - (1-y_train) * np.log(1-x)) / x.shape[0])
    

640/640 [==============================] - 3s 4ms/step - loss: 0.5630
0 Fitted result:  tf.Tensor([2.7531958 2.728015 ], shape=(2,), dtype=float32)
0.5618305075703683
640/640 [==============================] - 2s 4ms/step - loss: 0.5627
1 Fitted result:  tf.Tensor([2.8046625 2.8564637], shape=(2,), dtype=float32)
0.5616583040759288
640/640 [==============================] - 2s 3ms/step - loss: 0.5627
2 Fitted result:  tf.Tensor([2.8518248 2.9732978], shape=(2,), dtype=float32)
0.5615213488237641
640/640 [==============================] - 2s 3ms/step - loss: 0.5626
3 Fitted result:  tf.Tensor([2.9090626 3.0440218], shape=(2,), dtype=float32)
0.5614399866945169
640/640 [==============================] - 3s 4ms/step - loss: 0.5625
4 Fitted result:  tf.Tensor([2.9501114 3.0939536], shape=(2,), dtype=float32)
0.5613944166108559
640/640 [==============================] - 3s 4ms/step - loss: 0.5622
5 Fitted result:  tf.Tensor([2.9705653 3.1630957], shape=(2,), dtype=float32)
0.561337219037493